In [1]:
# define function to load train, test, and validation datasets
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob

def load_dataset(path):
    data = load_files(path)
    dog_files = np.array(data['filenames'])
    dog_targets = np_utils.to_categorical(np.array(data['target']), 133)
    return dog_files, dog_targets

# load train, test, and validation datasets
train_files, train_targets = load_dataset('dog_images/train')
valid_files, valid_targets = load_dataset('dog_images/valid')
test_files, test_targets = load_dataset('dog_images/test')


Using TensorFlow backend.


In [2]:
#dog breed predictor
### TODO: Obtain bottleneck features from another pre-trained CNN.
import numpy as np 

bottleneck_features = np.load('bottleneck_features/DogVGG19Data.npz')
train_VGG19 = bottleneck_features['train']
valid_VGG19 = bottleneck_features['valid']
test_VGG19 = bottleneck_features['test']

In [3]:
### TODO: Define your architecture.
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential


VGG19_model = Sequential()

VGG19_model.add(Dropout(0.3, input_shape=train_VGG19.shape[1:]))
VGG19_model.add(Dense(133, activation='relu'))
VGG19_model.add(Dropout(0.3))
VGG19_model.add(GlobalAveragePooling2D())
VGG19_model.add(Dense(133, activation='softmax'))

VGG19_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout_1 (Dropout)          (None, 7, 7, 512)         0         
_________________________________________________________________
dense_1 (Dense)              (None, 7, 7, 133)         68229     
_________________________________________________________________
dropout_2 (Dropout)          (None, 7, 7, 133)         0         
_________________________________________________________________
global_average_pooling2d_1 ( (None, 133)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 133)               17822     
Total params: 86,051
Trainable params: 86,051
Non-trainable params: 0
_________________________________________________________________


In [4]:
### TODO: Compile the model.

VGG19_model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [5]:
### TODO: Train the model.
from keras.callbacks import ModelCheckpoint  

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.VGG19.hdf5', 
                               verbose=1, save_best_only=True)

VGG19_model.fit(train_VGG19, train_targets, 
          validation_data=(valid_VGG19, valid_targets),
          epochs=20, batch_size=20, callbacks=[checkpointer], verbose=1)

Train on 6680 samples, validate on 835 samples
Epoch 1/20
6680/6680 [==============================] - 2s 347us/step - loss: 3.7944 - accuracy: 0.3100 - val_loss: 1.4290 - val_accuracy: 0.6060

Epoch 00001: val_loss improved from inf to 1.42903, saving model to saved_models/weights.best.VGG19.hdf5
Epoch 2/20
6680/6680 [==============================] - 2s 313us/step - loss: 1.1827 - accuracy: 0.6597 - val_loss: 1.0303 - val_accuracy: 0.6874

Epoch 00002: val_loss improved from 1.42903 to 1.03027, saving model to saved_models/weights.best.VGG19.hdf5
Epoch 3/20
6680/6680 [==============================] - 2s 330us/step - loss: 0.8256 - accuracy: 0.7564 - val_loss: 0.9558 - val_accuracy: 0.7329

Epoch 00003: val_loss improved from 1.03027 to 0.95576, saving model to saved_models/weights.best.VGG19.hdf5
Epoch 4/20
6680/6680 [==============================] - 2s 328us/step - loss: 0.6332 - accuracy: 0.8061 - val_loss: 0.9026 - val_accuracy: 0.7234

Epoch 00004: val_loss improved from 0.9557

In [6]:
### TODO: Load the model weights with the best validation loss.
VGG19_model.load_weights('saved_models/weights.best.VGG19.hdf5')

In [7]:
### Export pickle file
import pickle

pickle.dump(VGG19_model, open("model.pkl","wb"))